In [1]:
import numpy as np 
from scipy import stats
import pymc3 as pm
import arviz as az
import theano
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
az.style.use("arviz-darkgrid")
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Exercises

## 1:
Rerun the first model using the petal length and then petal width variables. What
are the main differences in the results? How wide or narrow is the 95% HPD
interval in each case?

In [3]:
# Load data
iris = pd.read_csv('../data/iris.csv')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
#wrangle time
df = iris.query("species == ('setosa', 'versicolor')")
y_0 = pd.Categorical(df['species']).codes
x_n = 'petal_length'
x_0 = df[x_n].values
x_c = x_0 - x_0.mean()

In [5]:
#petal length
with pm.Model() as model_1:
    α = pm.Normal('α', mu=0, sd=10)
    β = pm.Normal('β', mu=0, sd=10)
    μ = α + pm.math.dot(x_c, β)
    θ = pm.Deterministic('θ', pm.math.sigmoid(μ))
    bd = pm.Deterministic('bd', -α/β)
    yl = pm.Bernoulli('yl', p=θ, observed=y_0)
    trace_1 = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [β, α]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 21 seconds.
There were 165 divergences after tuning. Increase `target_accept` or reparameterize.
There were 99 divergences after tuning. Increase `target_accept` or reparameterize.
There were 107 divergences after tuning. Increase `target_accept` or reparameterize.
There were 129 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


In [6]:
#petal width
x_n = 'petal_width'
x_0 = df[x_n].values
x_c = x_0 - x_0.mean()

with pm.Model() as model_2:
    α = pm.Normal('α', mu=0, sd=10)
    β = pm.Normal('β', mu=0, sd=10)
    μ = α + pm.math.dot(x_c, β)
    θ = pm.Deterministic('θ', pm.math.sigmoid(μ))
    bd = pm.Deterministic('bd', -α/β)
    yl = pm.Bernoulli('yl', p=θ, observed=y_0)
    trace_2 = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [β, α]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 18 seconds.
There were 9 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


In [7]:
az.summary(trace_1)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
α,3.723,4.306,-3.389,12.295,0.157,0.116,753.0,695.0,860.0,907.0,1.0
β,13.854,5.701,4.640,24.110,0.196,0.139,845.0,845.0,859.0,1053.0,1.0
θ[0],0.000,0.001,0.000,0.001,0.000,0.000,1188.0,992.0,1004.0,1272.0,1.0
θ[1],0.000,0.001,0.000,0.001,0.000,0.000,1188.0,992.0,1004.0,1272.0,1.0
θ[2],0.000,0.001,0.000,0.001,0.000,0.000,1146.0,987.0,990.0,1255.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
θ[96],1.000,0.000,1.000,1.000,0.000,0.000,1862.0,1861.0,863.0,1227.0,1.0
θ[97],1.000,0.000,1.000,1.000,0.000,0.000,1952.0,1951.0,869.0,1217.0,1.0
θ[98],0.904,0.179,0.531,1.000,0.004,0.003,2051.0,1980.0,882.0,903.0,1.0
θ[99],1.000,0.001,1.000,1.000,0.000,0.000,1777.0,1775.0,861.0,1277.0,1.0


In [9]:
az.summary(trace_2)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
α,0.617,1.774,-2.734,4.033,0.052,0.038,1170.0,1074.0,1180.0,1454.0,1.01
β,17.898,5.384,8.855,27.993,0.159,0.115,1140.0,1092.0,1219.0,1546.0,1.00
θ[0],0.001,0.003,0.000,0.005,0.000,0.000,1308.0,813.0,1348.0,1101.0,1.00
θ[1],0.001,0.003,0.000,0.005,0.000,0.000,1308.0,813.0,1348.0,1101.0,1.00
θ[2],0.001,0.003,0.000,0.005,0.000,0.000,1308.0,813.0,1348.0,1101.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...
θ[96],0.999,0.003,0.996,1.000,0.000,0.000,1474.0,1474.0,1286.0,1646.0,1.00
θ[97],0.999,0.003,0.996,1.000,0.000,0.000,1474.0,1474.0,1286.0,1646.0,1.00
θ[98],0.989,0.019,0.958,1.000,0.000,0.000,2092.0,2092.0,1391.0,1424.0,1.01
θ[99],0.999,0.003,0.996,1.000,0.000,0.000,1474.0,1474.0,1286.0,1646.0,1.00


In [ ]:
## Exercise 2
